# 인구 이동 OD 시각화

## 기본 설정

In [ ]:
## 모든 변수 삭제
from IPython import get_ipython
get_ipython().magic('reset -sf')


#기본 라이브러리 로드
import os
import pandas as pd
import pydeck as pdk
import geopandas as gpd
from pyproj import Transformer
import re


In [122]:

#폴더 설정 필요
mainFolder =  "./"

#한글 폰트를 위한 세팅
from matplotlib import font_manager, rc
font_path = os.path.join(mainFolder,"ref","Pretendard-Bold.ttf") 
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

#결과 저장 폴더
writeFolder = os.path.join(mainFolder,"result")  # 파일 경로

#셰이더 패치 파일
deck_index_path = os.path.join(mainFolder,"ref/deck_index.js") 

## 데이터 읽기

In [130]:
useCols = list(range(41)) #읽어들일 열 번호
colNames = ['dessido', 'dessgg', 'desemd','year','month','day','orisido', 'orisgg','oriemd', 'motv',
            'rel00', 'age00', 'gender00', 'rel01', 'age01', 'gender01',
            'rel02', 'age02', 'gender02','rel03', 'age03', 'gender03',
            'rel04', 'age04', 'gender04','rel05', 'age05', 'gender05',
            'rel06', 'age06', 'gender06','rel07', 'age07', 'gender07',
            'rel08', 'age08', 'gender08','rel09', 'age09', 'gender09',
            'serial'] 
colTypesArr =['str', 'str','str','str','str','str','str','str','str','int64',
            'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 
            'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 
            'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 
            'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 
            'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 
            'int64']
colTypes = dict(zip(colNames, colTypesArr))   
fileName = os.path.join(mainFolder, "data","202202.txt")
mig_df = pd.read_csv(fileName, header=None,  na_values='',
                        usecols=useCols, names=colNames, dtype=colTypes, sep=';')
mig_df.head()

,dessido,dessgg,desemd,year,month,day,orisido,orisgg,oriemd,motv,...,rel07,age07,gender07,rel08,age08,gender08,rel09,age09,gender09,serial
0,48,250,62000,2022,02,01,26,320,54300,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529640
1,48,250,62000,2022,02,01,28,260,54300,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529637
2,48,250,62000,2022,02,01,29,170,52000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529638
3,48,250,62000,2022,02,01,48,121,54000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529641
4,48,250,62000,2022,02,01,48,123,52000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529639


## 전처리 1 : 기본 전처리

In [131]:
#출발지(전출),도착지(전입), 날짜 만들기
mig_df['oricd'] = mig_df['orisido'] + mig_df['orisgg'] + mig_df['oriemd']
mig_df['descd'] = mig_df['dessido'] + mig_df['dessgg'] + mig_df['desemd']
mig_df['yyyymmdd'] = mig_df['year'] + mig_df['month'] + mig_df['day']

mig_df.head()

,dessido,dessgg,desemd,year,month,day,orisido,orisgg,oriemd,motv,...,rel08,age08,gender08,rel09,age09,gender09,serial,oricd,descd,yyyymmdd
0,48,250,62000,2022,02,01,26,320,54300,1,...,NaN,NaN,NaN,NaN,NaN,NaN,529640,2632054300,4825062000,20220201
1,48,250,62000,2022,02,01,28,260,54300,2,...,NaN,NaN,NaN,NaN,NaN,NaN,529637,2826054300,4825062000,20220201
2,48,250,62000,2022,02,01,29,170,52000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,529638,2917052000,4825062000,20220201
3,48,250,62000,2022,02,01,48,121,54000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,529641,4812154000,4825062000,20220201
4,48,250,62000,2022,02,01,48,123,52000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,529639,4812352000,4825062000,20220201


In [132]:
dfs = []

# 추출하고 이름을 변경하는 과정을 for 루프로
for i in range(10):
    gender_col = f'gender0{i}'
    age_col = f'age0{i}'
    rel_col = f'rel0{i}'
    
    df_temp = mig_df[['yyyymmdd', 'oricd', 'descd', gender_col, age_col, rel_col]]
    df_temp = df_temp.rename(columns={gender_col: 'gender', age_col: 'age', rel_col: 'rel'})
    dfs.append(df_temp)

# 모든 데이터프레임을 합치기
mig_df = pd.concat(dfs, axis=0)

# NaN 값 제거 및 데이터 타입 변경
mig_df = mig_df.dropna(subset=['age', 'gender', 'rel'])
mig_df['age'] = mig_df['age'].astype('int64')
mig_df['gender'] = mig_df['gender'].astype('int64')
mig_df['rel'] = mig_df['rel'].astype('int64')

mig_df.head()


,yyyymmdd,oricd,descd,gender,age,rel
0,20220201,2632054300,4825062000,1,27,1
1,20220201,2826054300,4825062000,2,23,3
2,20220201,2917052000,4825062000,1,25,3
3,20220201,4812154000,4825062000,1,32,1
4,20220201,4812352000,4825062000,1,27,1


## 행정동 중심점을 읽고 좌표계 변환

In [136]:
#데이터 읽기
fileName = os.path.join(mainFolder, "ref","coordinate_UTMK_이름포함.tsv")
coord_df = pd.read_csv(fileName, header=0, sep='\t')
coord_df['ADMCD'] = coord_df['ADMCD'].astype('str')
coord_df.head()

# UTMK에서 WGS84로 좌표 변환
transformer = Transformer.from_crs("EPSG:5179", "EPSG:4326", always_xy=True)
coord_df['lon'], coord_df['lat'] = transformer.transform(coord_df['X'].values, coord_df['Y'].values)
coord_df.head()

#merge 준비
ori_coord = coord_df[['ADMCD', 'ADMNM', 'lon', 'lat']]
ori_coord.columns = ['oricd', 'orinm', 'orix', 'oriy']

des_coord = coord_df[['ADMCD', 'ADMNM', 'lon', 'lat']]
des_coord.columns = ['descd', 'desnm', 'desx', 'desy']

## 전처리 2 : 분석 목적에 따라 필요한 행만 추출/집계


In [151]:
# 다른 시군구 이동만 추출
filtered_df = mig_df.copy()

# 다른 시도간 이동만
#filtered_df = mig_df[mig_df['oricd'].str[:2] !=  mig_df['descd'].str[:2]]

# age가 20세 미만인 행만 추출
filtered_df = filtered_df[(filtered_df['age'] < 20) & (filtered_df['age'] >=15)]

# 행정동 내 이동은 제외
filtered_df = filtered_df[filtered_df['oricd'] != filtered_df['descd']]

# oricd와 descd 기준으로 groupby하고 od pair에 따른 이동 인원 세기
filtered_df = filtered_df.groupby(['oricd', 'descd']).size().reset_index(name='count')
filtered_df = filtered_df.sort_values(by='count', ascending=False)
filtered_df

,oricd,descd,count
20608,4420025300,4420033000,28
20388,4413356700,4420033000,21
8804,2826053700,2826054200,16
3577,1168060000,1168061000,16
12190,4111368000,4111369000,16
...,...,...,...
11881,3611055500,3611057000,1
11880,3611055500,3611056000,1
11879,3611055500,3611055600,1
2568,1153079000,1156053500,1


In [152]:
#출발 도착지에 좌표 결합
od_df = filtered_df.merge(ori_coord, how='left', on=['oricd'])
od_df = od_df.merge(des_coord, how='left', on=['descd'])
od_df.head()

,oricd,descd,count,orinm,orix,oriy,desnm,desx,desy
0,4420025300,4420033000,28,충청남도 아산시 배방읍,127.065952,36.752024,충청남도 아산시 탕정면,127.071781,36.807783
1,4413356700,4420033000,21,충청남도 천안시 서북구 불당2동,127.103392,36.812806,충청남도 아산시 탕정면,127.071781,36.807783
2,2826053700,2826054200,16,인천광역시 서구 청라2동,126.631636,37.534671,인천광역시 서구 가정1동,126.669214,37.526459
3,1168060000,1168061000,16,서울특별시 강남구 대치1동,127.059052,37.493333,서울특별시 강남구 대치2동,127.067349,37.500214
4,4111368000,4111369000,16,경기도 수원시 권선구 권선2동,127.024524,37.244951,경기도 수원시 권선구 곡선동,127.032182,37.239185


## 함수 정의

In [146]:
def patch_shader_in_html(writename) :
       
    # Step 1: 지정한 HTML 파일 읽기 with utf-8 encoding
    with open(writename, "r", encoding="utf-8") as file:
        html_content = file.read()

    # 주어진 조건을 충족하는 정규 표현식을 생성합니다.
    pattern = r"<script src='https:[^']*?@deck\.gl[^']*?index\.js'><\/script>"

    # 해당 패턴을 찾아서 <script></script>로 바꿉니다.
    html_content = re.sub(pattern, '<script></script>', html_content)
    
    # Step 4: deck_index.js 파일 내용 읽기 with utf-8 encoding
    with open(deck_index_path, "r", encoding="utf-8") as file:
        js_content = file.read()

    # <script></script> 태그 사이에 js 내용 삽입
    html_content = html_content.replace("<script></script>", f"<script>{js_content}</script>")

    # Step 5: 변경된 내용으로 HTML 파일 다시 저장 with utf-8 encoding
    with open(writename, "w", encoding="utf-8") as file:
        file.write(html_content)


In [147]:
def make_od_rank_map(od_df, origin_field, destination_field, value_field,
 line_origin_color, line_destination_color, line_width, 
 write_name,  coord_df, coord_df_name, coord_df_lon, coord_df_la, rank_show_num = 100) :

    sido_geojson =  gpd.read_file(os.path.join(mainFolder, "ref","sido.geojson"))

    #'rank' 열 생성 및 순위 매기기
    od_df['rank'] = od_df[value_field].rank(method='min', ascending=False).astype(int)
    od_df = od_df[od_df['rank'] <= rank_show_num]
    od_df = od_df.sort_values(by='rank', ascending=True)

    #최대값을 통해 두께 0~1.0 으로 구하기
    #최대값 구한다.
    max_value = od_df[value_field].max()
    print("max_value: ",max_value)
    od_df['width'] = od_df[value_field] / max_value

    #tooltip 만들기
    od_df['od_info'] = "순위 : " + od_df['rank'].astype(str) + "\n" + od_df[origin_field] + "➜" + od_df[destination_field] +"\n이동 인구 : " + od_df[value_field].astype(int).astype(str)

    # 표시할 부분만 emd_point_tsv 필터링하기
    unique_values = pd.concat([od_df[origin_field], od_df[destination_field]]).unique()
    filtered_emd = coord_df[coord_df[coord_df_name].isin(unique_values)]
    #filtered_emd.head()

    # GeoPandas의 GeoDataFrame을 PyDeck으로 변환
    layers = [

        pdk.Layer(
            'GeoJsonLayer',  # 사용할 레이어 타입
            sido_geojson,  # 데이터 소스
            opacity=1.0,  # 레이어의 투명도 
            stroked=True,  # 폴리곤 테두리선 그릴지 여부
            filled=True,  # 폴리곤 내부를 채울지 여부
            extruded=False,  # 3D 시각화 사용 여부
            wireframe=False,  # 와이어프레임 모드 사용 여부
            get_fill_color=[0, 0, 0, 255],  # 폴리곤 채우기 색상
            get_line_color=[120, 120, 120, 255],  # 폴리곤 테두리선 색상
            line_width=5,  # 선의 너비
            line_width_min_pixels=2,
        ),
        pdk.Layer(
            "ArcLayer",
            data= od_df,
            width_units =pdk.types.String("meters"),
            get_width = 'width' ,
            width_scale = line_width,
            get_source_position=["orix", "oriy"],
            get_target_position=["desx", "desy"],
            get_tilt=90,
            get_height = 0.1,
            get_source_color= line_origin_color,
            get_target_color= line_destination_color,
            pickable=True,
            auto_highlight=True,
            width_min_pixels =2,
            parameters = { 
                "depthTest": False,
                "blendFunc":[770, 1, 773, 1], #[GL.SRC_ALPHA, GL.ONE, GL.ONE_MINUS_DST_ALPHA, GL.ONE],
                "blendEquation": 32774, # GL.FUNC_ADD,
            },
            #tooltip={"text": "{od_nm}"},  # tooltip의 내용
        ),

        pdk.Layer(
            "TextLayer",
            filtered_emd,
            pickable=False,
            get_position= [coord_df_lon, coord_df_lat],
            get_text= coord_df_name,

            get_size= 150, #실제 지도상의 미터 단위. 
            #size_units =pdk.types.String("meters"),
            size_units =pdk.types.String("meters"),

            characterSet = pdk.types.String("auto"),
            fontFamily = pdk.types.String("Pretendard-Regular"),
            size_min_pixels =5,
            get_color=[255,255,255],
            get_angle=0,

            
            # Note that string constants in pydeck are explicitly passed as strings
            # This distinguishes them from columns in a data set
            get_text_anchor=pdk.types.String("middle"),
            get_alignment_baseline=pdk.types.String("center"),
            visible = True
        )

    ]

    # 맵 센터 정의 (서울시 중심)
    view_state = pdk.ViewState(latitude=37.5665, longitude=126.9780, zoom=8)

    # Pydeck Chart 생성
    map = pdk.Deck(layers=layers,
        initial_view_state=view_state,
        tooltip={"text": "{od_info}"},  # tooltip의 내용
        map_provider=None # 기본 배경지도 생략
    )

    # 맵 렌더링
    writename = os.path.join(writeFolder, write_name + ".html")
    map.to_html(writename, css_background_color="#222")

    patch_shader_in_html(writename) 

## 필요한 변수 셋팅 후 함수 호출

In [153]:
origin_field = "orinm"
destination_field = "desnm"
value_field = "count"
line_origin_color = [255, 200, 10, 5]
line_destination_color = [255, 50, 25, 255]
line_width = 1000
write_name = "19세미만 이동_2022"
rank_show_num = 200 

coord_df = coord_df
coord_df_name = 'ADMNM'
coord_df_lon = 'lon'
coord_df_lat = 'lat'

make_od_rank_map(od_df, origin_field, destination_field, value_field,
 line_origin_color, line_destination_color, line_width, 
 write_name, coord_df, coord_df_name, coord_df_lon, coord_df_lat, rank_show_num)
 

max_value:  28
